In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

from time import time
from utils.df_loader import load_german_df
from utils.preprocessing import min_max_scale_numerical, remove_missing_values, inverse_dummy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from alibi.explainers import CounterFactualProto, CounterFactual
from alibi_cf.utils import get_cat_vars_dict

tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


ModuleNotFoundError: No module named 'utils'

In [2]:
df, feature_names, numerical_cols, categorical_cols, columns_type, target_name, possible_outcomes = load_german_df()

In [3]:
scaled_df, scaler = min_max_scale_numerical(df, numerical_cols)

In [4]:
scaled_df.head(5)

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,N,< 0 DM,0.029412,critical account/ other credits existing (not ...,domestic appliances,0.050567,unknown/ no savings account,.. >= 7 years,1.000000,male : single,...,1.000000,real estate,0.857143,none,own,0.333333,skilled employee / official,0.0,"yes, registered under the customers name",yes
1,Y,0 <= ... < 200 DM,0.647059,existing credits paid back duly till now,domestic appliances,0.313690,... < 100 DM,1 <= ... < 4 years,0.333333,female : divorced/separated/married,...,0.333333,real estate,0.053571,none,own,0.000000,skilled employee / official,0.0,none,yes
2,N,no checking account,0.117647,critical account/ other credits existing (not ...,(vacation - does not exist?),0.101574,... < 100 DM,4 <= ... < 7 years,0.333333,male : single,...,0.666667,real estate,0.535714,none,own,0.000000,unskilled - resident,1.0,none,yes
3,N,< 0 DM,0.558824,existing credits paid back duly till now,radio/television,0.419941,... < 100 DM,4 <= ... < 7 years,0.333333,male : single,...,1.000000,if not A121 : building society savings agreeme...,0.464286,none,for free,0.000000,skilled employee / official,1.0,none,yes
4,Y,< 0 DM,0.294118,delay in paying off in the past,car (new),0.254209,... < 100 DM,1 <= ... < 4 years,0.666667,male : single,...,1.000000,unknown / no property,0.607143,none,for free,0.333333,skilled employee / official,1.0,none,yes


In [5]:
dummy_df = pd.get_dummies(scaled_df, columns=  [ col for col in categorical_cols if col != target_name])

In [6]:
### We should have this amount of input features.
sum([len(scaled_df[col].unique()) for col in categorical_cols if col != target_name]) + len(numerical_cols)

61

In [7]:
# enconded_df, encoder_dict = label_encode(scaled_df, categorical_cols)

In [8]:
cat_to_ohe_cat = {}
for c_col in categorical_cols:
    if c_col != target_name:
        cat_to_ohe_cat[c_col] = [ ohe_col for ohe_col in dummy_df.columns if ohe_col.startswith(c_col) and ohe_col != target_name]

In [9]:
ohe_feature_names = [ col for col in dummy_df.columns if col != target_name]

In [10]:
dummy_df.head(5)

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status_0 <= ... < 200 DM,account_check_status_< 0 DM,...,housing_own,housing_rent,job_management/ self-employed/ highly qualified employee/ officer,job_skilled employee / official,job_unemployed/ unskilled - non-resident,job_unskilled - resident,telephone_none,"telephone_yes, registered under the customers name",foreign_worker_no,foreign_worker_yes
0,N,0.029412,0.050567,1.000000,1.000000,0.857143,0.333333,0.0,0,1,...,1,0,0,1,0,0,0,1,0,1
1,Y,0.647059,0.313690,0.333333,0.333333,0.053571,0.000000,0.0,1,0,...,1,0,0,1,0,0,1,0,0,1
2,N,0.117647,0.101574,0.333333,0.666667,0.535714,0.000000,1.0,0,0,...,1,0,0,0,0,1,1,0,0,1
3,N,0.558824,0.419941,0.333333,1.000000,0.464286,0.000000,1.0,0,1,...,0,0,0,1,0,0,1,0,0,1
4,Y,0.294118,0.254209,0.666667,1.000000,0.607143,0.333333,1.0,0,1,...,0,0,0,1,0,0,1,0,0,1


In [11]:
inverse_dummy(dummy_df, cat_to_ohe_cat).head(5)

,default,duration_in_month,credit_amount,installment_as_income_perc,present_res_since,age,credits_this_bank,people_under_maintenance,account_check_status,credit_history,...,savings,present_emp_since,personal_status_sex,other_debtors,property,other_installment_plans,housing,job,telephone,foreign_worker
0,N,0.029412,0.050567,1.000000,1.000000,0.857143,0.333333,0.0,< 0 DM,critical account/ other credits existing (not ...,...,unknown/ no savings account,.. >= 7 years,male : single,none,real estate,none,own,skilled employee / official,"yes, registered under the customers name",yes
1,Y,0.647059,0.313690,0.333333,0.333333,0.053571,0.000000,0.0,0 <= ... < 200 DM,existing credits paid back duly till now,...,... < 100 DM,1 <= ... < 4 years,female : divorced/separated/married,none,real estate,none,own,skilled employee / official,none,yes
2,N,0.117647,0.101574,0.333333,0.666667,0.535714,0.000000,1.0,no checking account,critical account/ other credits existing (not ...,...,... < 100 DM,4 <= ... < 7 years,male : single,none,real estate,none,own,unskilled - resident,none,yes
3,N,0.558824,0.419941,0.333333,1.000000,0.464286,0.000000,1.0,< 0 DM,existing credits paid back duly till now,...,... < 100 DM,4 <= ... < 7 years,male : single,guarantor,if not A121 : building society savings agreeme...,none,for free,skilled employee / official,none,yes
4,Y,0.294118,0.254209,0.666667,1.000000,0.607143,0.333333,1.0,< 0 DM,delay in paying off in the past,...,... < 100 DM,1 <= ... < 4 years,male : single,none,unknown / no property,none,for free,skilled employee / official,none,yes


In [12]:
from sklearn.preprocessing import LabelEncoder

target_label_encoder = LabelEncoder()
dummy_df[target_name] = target_label_encoder.fit_transform(dummy_df[target_name])

dummy_df= dummy_df[ohe_feature_names + [target_name]]

In [13]:
train_df, test_df = train_test_split(dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [14]:
X_train = np.array(train_df[ohe_feature_names])
y_train = np.array(train_df[target_name])
X_test = np.array(test_df[ohe_feature_names])
y_test = np.array(test_df[target_name])

In [15]:
### Train
nn = model= tf.keras.models.Sequential(
            [
                tf.keras.layers.Dense(24,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(12,activation='relu'),
                tf.keras.layers.Dense(1),
                tf.keras.layers.Activation(tf.nn.sigmoid),
            ]
        )
nn.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train, batch_size=64, epochs=20, shuffle=True)

models = {
    "dt": DecisionTreeClassifier().fit(X_train,y_train),
    "rfc": RandomForestClassifier().fit(X_train,y_train),
    "nn": nn,
}

pickle.dump(models['dt'], open('./saved_models/dt.p', 'wb'))
pickle.dump(models['rfc'], open('./saved_models/rfc.p', 'wb'))
models['nn'].save('./saved_models/nn.h5',overwrite=True)


Train on 800 samples
Epoch 1/20
800/800 [==============================] - 0s 80us/sample - loss: 0.6960 - acc: 0.4950
Epoch 2/20
800/800 [==============================] - 0s 9us/sample - loss: 0.6707 - acc: 0.7013
Epoch 3/20
800/800 [==============================] - 0s 9us/sample - loss: 0.6473 - acc: 0.7013
Epoch 4/20
800/800 [==============================] - 0s 8us/sample - loss: 0.6159 - acc: 0.7013
Epoch 5/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5928 - acc: 0.7013
Epoch 6/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5792 - acc: 0.7013
Epoch 7/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5607 - acc: 0.7013
Epoch 8/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5430 - acc: 0.7050
Epoch 9/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5218 - acc: 0.7487
Epoch 10/20
800/800 [==============================] - 0s 7us/sample - loss: 0.5061 - acc: 0.7550
Epoch 1

In [16]:
### Load
models = {}
models['dt'] = pickle.load(open('./saved_models/dt.p', 'rb'))
models['rfc'] = pickle.load(open('./saved_models/rfc.p', 'rb'))
models['nn'] = tf.keras.models.load_model('./saved_models/nn.h5')

## Initialise NN output shape as (None, 1) for tensorflow.v1
models['nn'].predict(np.zeros((2, X_train.shape[-1])))

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


array([[0.3854977],
       [0.3854977]], dtype=float32)

In [17]:
example_data = X_test[0, :].reshape(1,-1)

dt_pred = models['dt'].predict(example_data)[0]
rfc_pred = models['rfc'].predict(example_data)[0]
nn_pred = models['nn'].predict(example_data)[0][0]

print(f"DT [{dt_pred}], RFC [{rfc_pred}], NN [{nn_pred}]")

DT [1], RFC [1], NN [0.8426796793937683]


# Alibi

## 1. Counterfactual Prototype

In [18]:
cat_vars_dict = get_cat_vars_dict(scaled_df, categorical_cols, feature_names, target_name)

In [19]:
cat_vars_dict

{0: 4,
 2: 5,
 3: 10,
 5: 5,
 6: 5,
 8: 4,
 9: 3,
 11: 4,
 13: 3,
 14: 3,
 16: 4,
 18: 2,
 19: 2}

In [20]:
cat_feature_names = [ col for col in categorical_cols if col != target_name ] 

In [21]:
cat_vars_idx_info = []

for cat_col in cat_feature_names:
    num_unique_v = len([ col for col in train_df.columns if col.startswith(f"{cat_col}_")])
    first_index = min([ list(train_df.columns).index(col) for col in train_df.columns if col.startswith(f"{cat_col}_")])
    
    cat_vars_idx_info.append({
        "col": cat_col,
        "num_unique_v": num_unique_v,
        "first_index": first_index
    })

In [22]:
cat_vars_idx_info

[{'col': 'account_check_status', 'num_unique_v': 4, 'first_index': 7},
 {'col': 'credit_history', 'num_unique_v': 5, 'first_index': 11},
 {'col': 'purpose', 'num_unique_v': 10, 'first_index': 16},
 {'col': 'savings', 'num_unique_v': 5, 'first_index': 26},
 {'col': 'present_emp_since', 'num_unique_v': 5, 'first_index': 31},
 {'col': 'personal_status_sex', 'num_unique_v': 4, 'first_index': 36},
 {'col': 'other_debtors', 'num_unique_v': 3, 'first_index': 40},
 {'col': 'property', 'num_unique_v': 4, 'first_index': 43},
 {'col': 'other_installment_plans', 'num_unique_v': 3, 'first_index': 47},
 {'col': 'housing', 'num_unique_v': 3, 'first_index': 50},
 {'col': 'job', 'num_unique_v': 4, 'first_index': 53},
 {'col': 'telephone', 'num_unique_v': 2, 'first_index': 57},
 {'col': 'foreign_worker', 'num_unique_v': 2, 'first_index': 59}]

In [23]:
cat_vars_ohe = {}

for idx_info in cat_vars_idx_info:
    cat_vars_ohe[idx_info['first_index']] = idx_info['num_unique_v']


In [24]:
from alibi_cf import AlibiBinaryPredictWrapper

In [25]:
alibi_wrapped = {
    'dt': AlibiBinaryPredictWrapper(models['dt']),
    'rfc': AlibiBinaryPredictWrapper(models['rfc']),
    'nn': AlibiBinaryPredictWrapper(models['nn']),
}

In [26]:
feature_range = (np.ones((1, len(feature_names))), np.zeros((1, len(feature_names))))

In [27]:
cf_p_dict = {}

for k in alibi_wrapped.keys():
    cf_p_dict[k] = CounterFactualProto(
                                alibi_wrapped[k].predict,
                                example_data.shape,
                                cat_vars=cat_vars_ohe,
                                feature_range=feature_range,
                                max_iterations=500,
                                ohe=True,
                                )

    cf_p_dict[k].fit(X_train)
    
""

''

In [ ]:
print(cat_vars_ohe)
print(example_data.shape)
print(feature_range)

In [28]:
num_instances = 5
num_cf_per_instance = 1

In [29]:
results = {}
for k in cf_p_dict.keys():
    results[k] = []
    print(f"Finding counterfactual for {k}")
    for idx, instance in enumerate(X_test[0:num_instances]):
        print(f"instance {idx}")
        example = instance.reshape(1, -1)
        for num_cf in range(num_cf_per_instance):
            print(f"CF {num_cf}")
            start_t = time()
            exp = cf_p_dict[k].explain(example)
            end_t = time ()
            running_time = end_t - start_t

            if k=='nn':
                prediction = target_label_encoder.inverse_transform((models[k].predict(example)[0]> 0.5).astype(int))[0]
            else:
                prediction = target_label_encoder.inverse_transform(models[k].predict(example))[0]

            if (not exp.cf is None) and (len(exp.cf) > 0):
                print("Found CF")
                if k == 'nn':
                    cf = inverse_dummy(pd.DataFrame(exp.cf['X'], columns=ohe_feature_names), cat_to_ohe_cat)
                    cf.loc[0, target_name] = target_label_encoder.inverse_transform([exp.cf['class']])[0]
                else:
                    cf = inverse_dummy(pd.DataFrame(exp.cf['X'], columns=ohe_feature_names), cat_to_ohe_cat)
                    cf.loc[0, target_name] = target_label_encoder.inverse_transform([exp.cf['class']])[0]
                # print(exp.cf)
                # print(cf)
            else:
                print("CF not found")
                cf = None

            input_df = inverse_dummy(pd.DataFrame(example, columns=ohe_feature_names), cat_to_ohe_cat)
            input_df.loc[0, target_name] = prediction

            results[k].append({
                "input": input_df,
                "cf": cf,
                'exp': exp,
                "running_time": running_time,
                "ground_truth": target_label_encoder.inverse_transform([y_test[idx]])[0],
                "prediction": prediction,
            })

Finding counterfactual for dt
instance 0
CF 0
CF not found
instance 1
CF 0
Found CF
instance 2
CF 0


In [ ]:
all_df = {}

for k in results.keys():

    all_data = []

    for i in range(len(results[k])):
        final_df = pd.DataFrame([{}])

        scaled_input_df = results[k][i]['input'].copy(deep=True)
        origin_columns = [f"origin_input_{col}"  for col in scaled_input_df.columns]
        origin_input_df = scaled_input_df.copy(deep=True)
        scaled_input_df.columns = [f"scaled_input_{col}"  for col in scaled_input_df.columns]

        origin_input_df[numerical_cols] = scaler.inverse_transform(origin_input_df[numerical_cols])
        origin_input_df.columns = origin_columns

        final_df = final_df.join([scaled_input_df, origin_input_df])

        if not results[k][i]['cf'] is None:
            scaled_cf_df = results[k][i]['cf'].copy(deep=True)
            # scaled_cf_df.loc[0, target_name] = target_label_encoder.inverse_transform([scaled_cf_df.loc[0, target_name]])[0]
            origin_cf_columns = [f"origin_cf_{col}"  for col in scaled_cf_df.columns]
            origin_cf_df = scaled_cf_df.copy(deep=True)
            scaled_cf_df.columns = [f"scaled_cf_{col}"  for col in scaled_cf_df.columns]

            origin_cf_df[numerical_cols] = scaler.inverse_transform(origin_cf_df[numerical_cols])
            origin_cf_df.columns = origin_cf_columns

            final_df = final_df.join([scaled_cf_df, origin_cf_df])

        # final_df = final_df.join([scaled_input_df, origin_input_df, scaled_cf_df, origin_cf_df])
        final_df['running_time'] = results[k][i]['running_time']
        final_df['Found'] = "Y" if not results[k][i]['cf'] is None else "N"
        final_df['ground_truth'] = results[k][i]['ground_truth'] 
        final_df['prediction'] = results[k][i]['prediction'] 

        all_data.append(final_df)

    all_df[k] = pd.concat(all_data)

In [ ]:
for df_k in all_df.keys():
    all_df[df_k].to_csv(f"./results/proto_german_{df_k}_result.csv")